# Catatan yang Diulik

1. Modality Based, mempertimbangkan kolom rating, jumlah reviews, nama tempat, tipe wisata, Waktu buka, dan koordinat
2. Interkoneksi antar tempat wisata, dengan mempertimbangkan nilai di user. Nilai yang dimaksud misal user A pernah merating berapa di tempat A, lalu telusuri dia pernah merating dimana saja. Nanti dijadikan identitas user yang digunakan untuk rekomendasi item, tapi soft recom, tidak user based.

Catatan:
- Untuk koordinat bisa coba ulik rumus **Haversine**
- Poin ke 2, bisa pake konsep inputan baru ke model rekomendasi, jadi inputan nnti di cross calculation dari hasil rekomendasi dari poin 1.
- Waktu membuka website atau waktu komputer dijadikan pertimbangan rekomendasi dari nilai waktu buka di data item.

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from math import radians, sin, cos, sqrt, atan2
from datetime import datetime
from IPython.display import display

fixedDf = pd.read_csv('data/fixedData.csv')
fixedDf.info()

In [ ]:
fixedDf.head()

# Data Preparation

## Standarisasi dan Normalisasi

### Workday Timing

In [ ]:
fixedDf.workday_timing.unique()

## Rating

In [61]:
# Standarisasi Kolom Rating Menggunakan Z-score
scaler = StandardScaler()

# Data 'rating' akan distandarisasi
fixedDf['rating_scaled'] = scaler.fit_transform(fixedDf[['rating']])

## Types

# Model Development

### Perhitungan Haversine (Coordinates)

In [62]:
# Perhitungan jarak menggunakan rumus Haversine
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Radius bumi dalam kilometer
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c  # Hasil dalam kilometer

In [63]:
# 2. Buat dataframe untuk wisata yang mirip secara lokasi
def filter_by_coordinates(df, user_lat, user_lon, max_distance_km=10):
    df['distance'] = df['coordinates'].apply(lambda coord: haversine(user_lat, user_lon, *map(float, coord.split(','))))
    similar_places_df = df[df['distance'] <= max_distance_km]
    
    # Urutkan berdasarkan jarak terdekat
    return similar_places_df.sort_values(by='distance', ascending=True)

### Perhitungan Cosine Similarity

In [64]:
# Modifikasi search_by_name untuk mengurutkan berdasarkan rating dan Cosine Sim
def search_by_name(df, user_input):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(df['name'])
    user_input_tfidf = tfidf_vectorizer.transform([user_input])

    cosine_sim = cosine_similarity(user_input_tfidf, tfidf_matrix)
    df['similarity'] = cosine_sim[0]
    
    # Urutkan berdasarkan rating setelah similarity
    df = df.sort_values(by=['similarity', 'rating'], ascending=[False, False])
    
    return df

### Filterisasi Waktu Kerja (Working Day)

In [65]:
# Filter berdasarkan waktu operasional (working days)
def filter_by_workday(df, current_time):
    def is_open(workday_timing, current_time):
        if workday_timing == 'Not Present' or workday_timing == 'Closed':
            return False
        try:
            open_time, close_time = workday_timing.split('-')
            open_time = datetime.strptime(open_time.strip(), '%H.%M').time()
            close_time = datetime.strptime(close_time.strip(), '%H.%M').time()
            # Periksa apakah waktu sekarang berada dalam rentang waktu buka
            return open_time <= current_time <= close_time
        except:
            return False

    # Filter berdasarkan apakah tempat wisata sedang buka
    df['is_open'] = df['workday_timing'].apply(lambda x: is_open(x, current_time))
    
    return df[df['is_open'] == True]

### Filterisasi Tipe Wisata

In [66]:
# Filter berdasarkan tipe tempat wisata
def filter_by_type(df, selected_type):
    return df[df['types'].str.contains(selected_type, case=False, na=False)]

## Filterisasi Berdasarkan Jarak Terdekat


In [67]:
# Fungsi untuk mencari tempat wisata terdekat setelah memilih salah satu
def find_nearby_places(df, selected_place_lat, selected_place_lon, max_distance_km=10, top_n=5):
    # Hitung jarak dari tempat wisata yang dipilih
    df['distance_from_selected'] = df['coordinates'].apply(lambda coord: haversine(selected_place_lat, selected_place_lon, *map(float, coord.split(','))))
    
    # Filter tempat wisata yang berada dalam jarak tertentu
    nearby_df = df[df['distance_from_selected'] <= max_distance_km]
    
    # Urutkan berdasarkan jarak terdekat
    nearby_df = nearby_df.sort_values(by='distance_from_selected', ascending=True)
    
    # Kembalikan top-N hasil
    return nearby_df.head(top_n)

# Modifikasi fungsi utama untuk memberikan rekomendasi setelah memilih tempat
def recommend_nearby(df, selected_place_id, max_distance_km=10, top_n=5):
    # Ambil koordinat tempat wisata yang dipilih
    selected_place = df[df['place_id'] == selected_place_id].iloc[0]
    selected_place_lat, selected_place_lon = map(float, selected_place['coordinates'].split(','))

    # Cari tempat wisata terdekat
    nearby_recommendations = find_nearby_places(df, selected_place_lat, selected_place_lon, max_distance_km, top_n)
    
    return nearby_recommendations

## Perhitungan IMDB Weighted Rating

In [68]:
# Fungsi untuk menghitung weighted rating
def weighted_rating(x, m, C):
    v = x['reviews']  # Jumlah ulasan
    R = x['rating']   # Rating rata-rata tempat wisata
    # Hitung weighted rating
    return (v / (v + m) * R) + (m / (v + m) * C)

# 2. Fungsi untuk mengurutkan berdasarkan weighted rating
def sort_by_weighted_rating(df):
    # Tentukan m (jumlah ulasan minimum) dan C (rating rata-rata keseluruhan)
    C = df['rating'].mean()
    m = df['reviews'].quantile(0.75)  # Mengambil persentil 75% dari ulasan untuk threshold minimum

    # Hitung weighted rating untuk setiap baris
    df['weighted_rating'] = df.apply(lambda x: weighted_rating(x, m, C), axis=1)

    # Urutkan berdasarkan weighted rating tertinggi
    return df.sort_values(by='weighted_rating', ascending=False)

## Fungsi Rekomendasi

In [69]:
# Modifikasi fungsi rekomendasi untuk menggunakan weighted rating
def recommend(df, user_input=None, user_lat=None, user_lon=None, selected_type=None, max_distance_km=10, top_n=5):
    # Pencarian berdasarkan nama jika ada input pengguna
    if user_input:
        df = search_by_name(df, user_input)
    
    # Filter berdasarkan tipe tempat wisata jika diberikan
    if selected_type:
        df = filter_by_type(df, selected_type)
    
    # Urutkan berdasarkan weighted rating
    df = sort_by_weighted_rating(df)
    
    # Return hasil pencarian berdasarkan nama dan lokasi
    return df.head(top_n)

## Pencarian Dengan Search Field

In [ ]:
# Eksekusi pencarian pertama kali (sebelum memilih tempat)
user_lat = -7.774189
user_lon = 110.3647986
user_input = "Museum"
selected_type = "Museum"
max_distance_km = 0.2
top_n = 20

# Panggil fungsi rekomendasi awal
name_recommendations = recommend(fixedDf, user_input, user_lat, user_lon, selected_type, max_distance_km, top_n)

# Tampilkan hasil
print("Recommendations based on name search:")
name_recommendations

## Pencarian dari Tempat Yang Dipilih

In [71]:
# # Eksekusi pencarian berdasarkan tempat yang dipilih
# selected_place_id = 67  # Misalnya, pengguna memilih tempat dengan place_id 0
# max_distance_km = 5  # Jarak maksimal dari tempat yang dipilih
# top_n = 10  # Ambil 3 tempat terdekat

# # Periksa apakah rekomendasi pertama berisi kolom place_id
# if 'place_id' in name_recommendations.columns:
#     # Panggil fungsi rekomendasi untuk tempat terdekat
#     nearby_recommendations = recommend_nearby(name_recommendations, selected_place_id, max_distance_km, top_n)

#     # Tampilkan hasil rekomendasi terdekat menggunakan display()
#     print("\nNearby recommendations from the selected place:")
#     display(nearby_recommendations)  # Menggunakan display untuk menampilkan tabel dengan lebih baik
# else:
#     print("Place ID tidak ditemukan dalam hasil pencarian.")

## Model Evaluation

In [ ]:
from sklearn.metrics import mean_absolute_error

# Assuming 'rating' is the actual value and 'weighted_rating' is the predicted value
def calculate_mae(df):
    # Ensure that 'weighted_rating' is already computed
    if 'weighted_rating' not in df.columns:
        # Calculate 'weighted_rating' if not present
        C = df['rating'].mean()
        m = df['reviews'].quantile(0.75)
        df['weighted_rating'] = df.apply(lambda x: weighted_rating(x, m, C), axis=1)
    
    # Calculate MAE between actual 'rating' and 'weighted_rating'
    mae_value = mean_absolute_error(df['rating'], df['weighted_rating'])
    
    return mae_value

# Calculate the MAE for the dataset
mae = calculate_mae(fixedDf)

print("Mean Absolute Error (MAE) between rating and weighted rating:", mae)


In [ ]:
from sklearn.metrics import f1_score, confusion_matrix

# Step 1: Convert ratings to categories
def categorize_rating(rating):
    if rating <= 2.5:
        return 'Low'
    elif 2.5 < rating <= 4.0:
        return 'Medium'
    else:
        return 'High'

# Apply categorization to actual and predicted ratings
fixedDf['rating_category'] = fixedDf['rating'].apply(categorize_rating)
fixedDf['weighted_rating_category'] = fixedDf['weighted_rating'].apply(categorize_rating)

# Step 2: Calculate F1 Score
f1 = f1_score(fixedDf['rating_category'], fixedDf['weighted_rating_category'], average='weighted')

# Step 3: Calculate Confusion Matrix
conf_matrix = confusion_matrix(fixedDf['rating_category'], fixedDf['weighted_rating_category'])

# Print results
print("F1 Score:", f1)
print("Confusion Matrix:\n", conf_matrix)


## Coba

In [ ]:
# Contoh dataset riwayat rating user
# Kolom: user_id, place_id, rating
user_ratings = pd.DataFrame({
    'user_id': [1, 1, 1, 2, 2],
    'place_id': [10, 20, 30, 10, 40],
    'rating': [4, 5, 3, 5, 4]
})

# Step 1: Memeriksa tempat yang pernah dirating oleh user
def get_user_rated_places(user_id, ratings_df):
    user_rated_places = ratings_df[ratings_df['user_id'] == user_id]
    return user_rated_places

# Step 2: Menghitung kemiripan antar tempat berdasarkan rating
def calculate_similarity_based_on_ratings(place_id, ratings_df):
    # Membuat pivot table: baris = place, kolom = user, nilai = rating
    rating_pivot = ratings_df.pivot_table(index='place_id', columns='user_id', values='rating').fillna(0)
    
    # Menghitung cosine similarity antar tempat
    similarity_matrix = cosine_similarity(rating_pivot)
    
    # Menyusun hasil dalam dataframe
    similarity_df = pd.DataFrame(similarity_matrix, index=rating_pivot.index, columns=rating_pivot.index)
    
    # Mengurutkan tempat yang mirip dengan place_id yang dipilih
    similar_places = similarity_df[place_id].sort_values(ascending=False)
    return similar_places

# Step 3: Menggabungkan hasil rekomendasi awal dengan hasil dari rating user
def combine_recommendations(initial_recommendations, user_rated_places, ratings_df):
    combined_results = pd.DataFrame()
    
    for place_id in user_rated_places['place_id']:
        # Dapatkan tempat yang mirip berdasarkan rating user
        similar_places = calculate_similarity_based_on_ratings(place_id, ratings_df)
        
        # Gabungkan dengan hasil rekomendasi awal
        combined_results = pd.concat([combined_results, initial_recommendations[initial_recommendations['place_id'].isin(similar_places.index)]])
    
    # Menghapus duplikat dan mengurutkan berdasarkan relevansi
    combined_results = combined_results.drop_duplicates().sort_values(by='rating', ascending=False)
    
    return combined_results

# Step 4: Menjalankan kombinasi rekomendasi
user_id = 1  # Contoh user
user_rated_places = get_user_rated_places(user_id, user_ratings)
final_recommendations = combine_recommendations(name_recommendations, user_rated_places, user_ratings)

# Menampilkan hasil rekomendasi final
display(final_recommendations)
